In [ ]:
# NeurIPS 2025 - Google Code Golf Championship | Version Mark XII
import os, zipfile, shutil, sys, importlib.util, json, re, numpy, copy
# Golfed utility loader (l): loads task JSON data
C = '/kaggle/input/google-code-golf-2025/';
def l(t,c=C):
  if not t: return {"train": [], "test": [], "arc-gen": []};
  try:
      with open(c+f'task{t:03d}.json')as f:e=json.load(f);
  except FileNotFoundError: # Fallback path check
      c_alt = '/kaggle/input/google-code-golf-2025/data/';
      with open(c_alt+f'task{t:03d}.json')as f:e=json.load(f);
  return e
# Golfed Verification Function (v): returns non-zero (count of failed examples) if invalid
def v(t,p='/kaggle/working/task.py',M='p'):
  s=importlib.util.spec_from_file_location('T',p);
  m=sys.modules['T']=importlib.util.module_from_spec(s);
  s.loader.exec_module(m);
  if not hasattr(m,M) or not callable(getattr(m,M)):return 400
  P=getattr(m,M);e=l(t);r=0;
  for i in e.get('train',[])+e.get('test',[])+e.get('arc-gen',[]):
    try:
      o=P(copy.deepcopy(i['input']))
      o=json.dumps(o).replace('true','1').replace('false','0')
      if re.search(r'[^0-9,\[\\]\s\.]',o):raise Exception
      o=numpy.array(json.loads(o))
      if not numpy.array_equal(o,numpy.array(i['output'])):r+=1
    except: r+=1
  return r

# --- Main Ensemble Logic ---
P=["/kaggle/input/big-zippa","/kaggle/input/code-compression","/kaggle/input/neuroips-4-of-some-lessons-learned","/kaggle/input/neuroips-31-unsolved","/kaggle/input/source-code-for-full-400-solutions","/kaggle/input/400-task-with-smart-solution-search-verification","/kaggle/input/python-minifier-applied", "/kaggle/input/code-compression-900k-club", "/kaggle/input/new-super-golfed-solutions"]
S='/kaggle/working/submission';
os.makedirs(S,exist_ok=1); # <-- FLOW FIX: Ensure S exists first!

# --- Task 400 Targeted Injection (Minimal Code) ---
T400_CODE = "p=lambda g:g[:5][:5]" # 18 bytes (Slightly shorter version of extracting top-left 5x5)
T400_PATH = os.path.join(S, 'task400.py')
with open(T400_PATH, 'w', encoding='L1') as f:
    f.write(T400_CODE)
# The new script is now at T400_PATH, ready to be picked up by the ensemble search in the next step.

score=0
# 1. Extract solutions from all input paths
for i,p in enumerate(P):
  d=f'submission_{i}';os.makedirs(d,exist_ok=1)
  for z in [os.path.join(p,'submission.zip'), os.path.join(p,'submission/submission.zip')]:
    if os.path.exists(z):
      try:
          with zipfile.ZipFile(z,'r')as Z:Z.extractall(d)
      except: continue
    
# 2. Search for the smallest valid solution for each task
for t in range(1,401):
  T=f'task{t:03d}.py';
  min_s=None;best_f=None
  
  # Inject the custom file path into the search if it's the target task (t=400)
  injected_path = T400_PATH if t == 400 else None

  for i in range(len(P)):
    d=f'submission_{i}'
    # List all file paths to check, prioritizing the injected path if applicable
    search_paths = []
    if injected_path and f'submission_{i}' == 'submission_0': # If the custom file is in the search pool
        search_paths.append(injected_path)

    search_paths.extend([os.path.join(d,T), os.path.join(d,'submission',T), os.path.join(d,'submission/submission',T)])

    for f in search_paths:
        if os.path.exists(f):
            s=os.path.getsize(f)
            if min_s is None or s<min_s:
                shutil.copy(f,'/kaggle/working/task.py')
                if not v(t):
                    min_s=s
                    best_f=f
  
  if min_s is not None:
    shutil.copy(best_f,os.path.join(S,T))
    score+=max(1,2500-min_s)
  else:score+=0.001

print(f"LB Score: {score}")

# 3. Final Submission Zip Creation
Zf=f'{S}.zip';
with zipfile.ZipFile(Zf,'w',zipfile.ZIP_DEFLATED)as zf:
  c=0
  for t in range(1,401):
    T=f'task{t:03d}.py';s=os.path.join(S,T)
    if os.path.exists(s):zf.write(s,T);c+=1
print(f"Created submission zip with {c} tasks: {Zf}")